In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files

uploaded = files.upload()  
image_path = list(uploaded.keys())[0]

image = cv2.imread(image_path)
if image is None:
    raise ValueError("Изображение не найдено.")

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)

contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

min_area = 50
filtered_contours = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > min_area:
        filtered_contours.append(cnt)

contours = filtered_contours

if not contours:
    raise ValueError("Не нашлось контуров (или все отфильтрованы).")

object_data = []  # (cx, cy, area, contour)
for cnt in contours:
    area = cv2.contourArea(cnt)
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = 0, 0
    object_data.append((cx, cy, area, cnt))

num_objects = len(object_data)

object_data.sort(key=lambda x: x[2])  # сортируем по площади
smallest = object_data[0]
largest = object_data[-1]

(cx_small, cy_small, area_small, cnt_small) = smallest
(cx_large, cy_large, area_large, cnt_large) = largest

#Визуализация (рисуем контуры и подписи)
output_image = image.copy()

for (cx, cy, area, cnt) in object_data:
    # Рисуем контур
    cv2.drawContours(output_image, [cnt], -1, (0, 255, 0), 2)
    # Ставим точку в центре
    cv2.circle(output_image, (cx, cy), 5, (0, 0, 255), -1)
    # Подписываем координаты
    cv2.putText(output_image, f"({cx},{cy})", (cx+5, cy-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

# Подписи для самого большого/маленького
cv2.putText(output_image, "Smallest", (cx_small, cy_small - 15),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
cv2.putText(output_image, "Largest", (cx_large, cy_large - 15),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)

output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

# результат
plt.figure(figsize=(12, 6))
plt.subplot(1,2,1)
plt.title(f"Исходное (найдено объектов: {num_objects})")
plt.imshow(image_rgb)
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Результат обработки")
plt.imshow(output_image_rgb)
plt.axis("off")
plt.show()


print("Количество объектов:", num_objects)
print("Координаты (x, y) самого маленького объекта:", (cx_small, cy_small))
print("Координаты (x, y) самого большого объекта:", (cx_large, cy_large))
